In [1]:
# Setup

import pandas as pd
from pathlib import Path
import hvplot.pandas
import requests
import json
from pprint import pprint
from config import geoapify_key

# Import and read files

file = Path("data/geoapify_df.csv")
df = pd.read_csv(file, encoding="utf-8")

In [2]:
# Create Country DF to show Number and Percent of Growing, Declining, Stagnant, and Total Startups for each Country

# Calculate number and percentages of startups

number_growing = []
number_declining = []
number_stagnant = []
number_total = []

unique_countries = df["Country"].unique()
countries_list = unique_countries.tolist()

for country in countries_list:
    growing = 0
    declining = 0
    stagnant = 0
    total = 0
    for index, row in df.iterrows():
        if row["Country"] == country:
            total = total + 1
            if row["CAGR (%)"] > 0:
                growing = growing + 1
            elif row["CAGR (%)"] < 0:
                declining = declining + 1
            else:
                stagnant = stagnant + 1
    number_growing.append(growing)
    number_declining.append(declining)
    number_stagnant.append(stagnant)
    number_total.append(total)

country_df = pd.DataFrame({
    "Country": countries_list,
    "Number of Growing Startups": number_growing,
    "Number of Declining Startups": number_declining,
    "Number of Stagnant Startups": number_stagnant,
    "Total Number of Startups": number_total
})

country_df["Percent of Growing Startups"] = (country_df["Number of Growing Startups"]/country_df["Total Number of Startups"]) * 100
country_df["Percent of Declining Startups"] = (country_df["Number of Declining Startups"]/country_df["Total Number of Startups"]) * 100
country_df["Percent of Stagnant Startups"] = (country_df["Number of Stagnant Startups"]/country_df["Total Number of Startups"]) * 100
country_df["Percent of World's Startups"] = (country_df["Total Number of Startups"]/1207) * 100

# Get Lat and Lng of each Country using Geoapify API

latitude = []
longitude = []

for index, row in country_df.iterrows():

    # Set search parameters

    target_country = f"{row['Country']}"

    # Base url

    base_url = f"https://api.geoapify.com/v1/geocode/search?country={target_country}&format=json&apiKey={geoapify_key}"

    # Run request and get longtitude and latitude
    
    response = requests.get(base_url).json()
    try:
        lat = response["results"][0]["lat"]
        latitude.append(lat)
    except:
        latitude.append(None)
        
    try:
        lng = response["results"][0]["lon"]
        longitude.append(lng)
    except:
        longitude.append(None)

country_df["Lat"] = latitude
country_df["Lng"] = longitude

country_df

,Country,Number of Growing Startups,Number of Declining Startups,Number of Stagnant Startups,Total Number of Startups,Percent of Growing Startups,Percent of Declining Startups,Percent of Stagnant Startups,Percent of World's Startups,Lat,Lng
0,China,4,1,144,149,2.684564,0.671141,96.644295,12.344656,35.000074,104.999927
1,United States,48,15,518,581,8.261618,2.581756,89.156627,48.135874,39.783730,-100.445882
2,Australia,2,1,5,8,25.000000,12.500000,62.500000,0.662800,-24.776109,134.755000
3,United Kingdom,3,3,37,43,6.976744,6.976744,86.046512,3.562552,54.702354,-3.276575
4,India,7,5,47,59,11.864407,8.474576,79.661017,4.888152,22.351115,78.667743
5,Germany,2,1,23,26,7.692308,3.846154,88.461538,2.154101,51.163818,10.447831
6,Turkey,1,1,1,3,33.333333,33.333333,33.333333,0.248550,38.959759,34.924965
7,Seychelles,0,0,1,1,0.000000,0.000000,100.000000,0.082850,-4.657498,55.454015
8,Sweden,0,0,5,5,0.000000,0.000000,100.000000,0.414250,59.674971,14.520858
9,Mexico,0,0,8,8,0.000000,0.000000,100.000000,0.662800,23.658512,-102.007710


In [3]:
# Map visualization of Top 10 Countries with the greatest % of startups in the world

greatest_percent_startups = country_df.nlargest(10, "Percent of World's Startups")

# Plot Map

import holoviews as hv

map_greatest_percent_startups = greatest_percent_startups.hvplot.points("Lng", "Lat", geo=True, tiles="OSM", 
                                                                         size="Percent of World's Startups", color="Country", 
                                                                         width=1000, height=500, scale=10, alpha=0.8,
                                                                         hover_cols=["Percent of World's Startups"]).opts(
    title="Top 10 Countries with the Greatest Percent of All Unicorn Startups in the World")

# Save map

hv.save(map_greatest_percent_startups, "output_data/map_greatest_percent_startups.html")

map_greatest_percent_startups

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,Percent of World's Startups)

In [5]:
# Map visualization of Top 10 Countries with the greatest # of startups in the world

greatest_number_startups = country_df.nlargest(10, "Total Number of Startups")

# Plot Map

map_greatest_number_startups = greatest_number_startups.hvplot.points("Lng", "Lat", geo=True, tiles="OSM", 
                                                                         size="Total Number of Startups", color="Country", 
                                                                         width=1000, height=500, scale=3, alpha=0.8,
                                                                         hover_cols=["Total Number of Startups"]).opts(
    title="Top 10 Countries with the Greatest Number of Unicorn Startups in the World")

# Save map

hv.save(map_greatest_number_startups, "output_data/map_greatest_number_startups.html")

map_greatest_number_startups

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,Total Number of Startups)

In [6]:
# Map visualization of Top 10 Countries with the greatest # of growing startups and greatest # of declining startups

greatest_number_growing = country_df.nlargest(10, "Number of Growing Startups")
greatest_number_declining = country_df.nlargest(5, "Number of Declining Startups")

# Plot Map of Top 10 Countries with the greatest # of growing startups

map_greatest_number_growing = greatest_number_growing.hvplot.points("Lng", "Lat", geo=True, tiles="OSM", 
                                                                         size="Number of Growing Startups", color="green", 
                                                                         width=1000, height=500, scale=8, alpha=0.8,
                                                                         hover_cols=["Country", "Number of Growing Startups"])

# Plot Map of Top 5 Countries with the greatest # of declining startups

map_greatest_number_declining = greatest_number_declining.hvplot.points("Lng", "Lat", geo=True, tiles="OSM", 
                                                                         size="Number of Declining Startups", color="red", 
                                                                         width=1000, height=500, scale=8, alpha=0.8,
                                                                         hover_cols=["Country", "Number of Declining Startups"])

# Combine maps

map_greatest_number_growing_declining = map_greatest_number_growing * map_greatest_number_declining
map_greatest_number_growing_declining = map_greatest_number_growing_declining.opts(
    title="Top Countries with the Greatest Number of Growing or Declining Unicorn Startups")

# Save map

hv.save(map_greatest_number_growing_declining, "output_data/map_greatest_number_growing_declining.html")

map_greatest_number_growing_declining

:Overlay
   .WMTS.I    :WMTS   [Longitude,Latitude]
   .Points.I  :Points   [Lng,Lat]   (Number of Growing Startups,Country)
   .WMTS.II   :WMTS   [Longitude,Latitude]
   .Points.II :Points   [Lng,Lat]   (Number of Declining Startups,Country)

In [7]:
# Map visualization of Top 10 Countries with the greatest % of growing startups and greatest % of declining startups

greatest_percent_growing = country_df.nlargest(10, "Percent of Growing Startups")
greatest_percent_declining = country_df.nlargest(10, "Percent of Declining Startups")

# Plot Map of Top 10 Countries with the greatest % of growing startups

map_greatest_percent_growing = greatest_percent_growing.hvplot.points("Lng", "Lat", geo=True, tiles="OSM", 
                                                                         size="Percent of Growing Startups", color="green", 
                                                                         width=1000, height=500, scale=5, alpha=0.8,
                                                                         hover_cols=["Country", "Percent of Growing Startups"])

# Plot Map of Top 10 Countries with the greatest % of declining startups

map_greatest_percent_declining = greatest_percent_declining.hvplot.points("Lng", "Lat", geo=True, tiles="OSM", 
                                                                         size="Percent of Declining Startups", color="red", 
                                                                         width=1000, height=500, scale=5, alpha=0.8,
                                                                         hover_cols=["Country", "Percent of Declining Startups"])

# Combine maps

map_greatest_percent_growing_declining = map_greatest_percent_growing * map_greatest_percent_declining
map_greatest_percent_growing_declining = map_greatest_percent_growing_declining.opts(
    title="Top Countries with Greatest Percent of Growing or Declining Unicorn Startups\n(Percentages are calculated out of the total number of unicorn startups per country, not globally.)")

# Save map

hv.save(map_greatest_percent_growing_declining, "output_data/map_greatest_percent_growing_declining.html")

map_greatest_percent_growing_declining

:Overlay
   .WMTS.I    :WMTS   [Longitude,Latitude]
   .Points.I  :Points   [Lng,Lat]   (Percent of Growing Startups,Country)
   .WMTS.II   :WMTS   [Longitude,Latitude]
   .Points.II :Points   [Lng,Lat]   (Percent of Declining Startups,Country)

In [8]:
# Clean df

df = df.drop(["Unnamed: 0.1", "Unnamed: 0"], axis=1)
df = df.drop_duplicates(subset=["Company", "Date Joined"])

In [9]:
# Map visualization of Top 10 Startups with the greatest CAGR and lowest CAGR (by city)

top_startups = df.nlargest(10, "CAGR (%)")
bottom_startups = df.nsmallest(10, "CAGR (%)")

# Map plot

map_top_startups = top_startups.hvplot.points("Lng", "Lat", geo=True, tiles="OSM", 
                                              size="2024_Valuation", color="green", 
                                              width=1000, height=500, scale=5, alpha=0.5,
                                              hover_cols=["Company", "Country", "CAGR (%)"])

map_bottom_startups = bottom_startups.hvplot.points("Lng", "Lat", geo=True, tiles="OSM", 
                                                    size="2024_Valuation", color="red", 
                                                    width=1000, height=500, scale=5, alpha=0.5,
                                                    hover_cols=["Company", "Country", "CAGR (%)"])

# Combine maps

map_startups = map_top_startups * map_bottom_startups
map_startups = map_startups.opts(
    title="Top 10 Startups with the Greatest and Lowest CAGR")

# Save map

hv.save(map_startups, "output_data/map_top_startups.html")

map_startups

:Overlay
   .WMTS.I    :WMTS   [Longitude,Latitude]
   .Points.I  :Points   [Lng,Lat]   (2024_Valuation,Company,Country,CAGR (%))
   .WMTS.II   :WMTS   [Longitude,Latitude]
   .Points.II :Points   [Lng,Lat]   (2024_Valuation,Company,Country,CAGR (%))

In [4]:
# DF of Top 10 Countries with the greatest % of startups in the world

print("Top 10 Countries with the greatest % of startups in the world")

greatest_percent_startups

Top 10 Countries with the greatest % of startups in the world


,Country,Number of Growing Startups,Number of Declining Startups,Number of Stagnant Startups,Total Number of Startups,Percent of Growing Startups,Percent of Declining Startups,Percent of Stagnant Startups,Percent of World's Startups,Lat,Lng
1,United States,48,15,518,581,8.261618,2.581756,89.156627,48.135874,39.783730,-100.445882
0,China,4,1,144,149,2.684564,0.671141,96.644295,12.344656,35.000074,104.999927
4,India,7,5,47,59,11.864407,8.474576,79.661017,4.888152,22.351115,78.667743
3,United Kingdom,3,3,37,43,6.976744,6.976744,86.046512,3.562552,54.702354,-3.276575
5,Germany,2,1,23,26,7.692308,3.846154,88.461538,2.154101,51.163818,10.447831
13,France,0,2,21,23,0.000000,8.695652,91.304348,1.905551,46.603354,1.888334
11,Canada,0,0,18,18,0.000000,0.000000,100.000000,1.491301,61.066692,-107.991707
14,Israel,2,0,16,18,11.111111,0.000000,88.888889,1.491301,30.812425,34.859476
18,Brazil,0,1,15,16,0.000000,6.250000,93.750000,1.325601,-10.333333,-53.200000
19,South Korea,2,0,11,13,15.384615,0.000000,84.615385,1.077051,36.638392,127.696119


In [12]:
# DF of Top 10 Countries with the greatest # of startups in the world

print("Top 10 Countries with the greatest # of startups in the world")

greatest_number_startups

Top 10 Countries with the greatest # of startups in the world


,Country,Number of Growing Startups,Number of Declining Startups,Number of Stagnant Startups,Total Number of Startups,Percent of Growing Startups,Percent of Declining Startups,Percent of Stagnant Startups,Percent of World's Startups,Lat,Lng
1,United States,48,15,518,581,8.261618,2.581756,89.156627,48.135874,39.783730,-100.445882
0,China,4,1,144,149,2.684564,0.671141,96.644295,12.344656,35.000074,104.999927
4,India,7,5,47,59,11.864407,8.474576,79.661017,4.888152,22.351115,78.667743
3,United Kingdom,3,3,37,43,6.976744,6.976744,86.046512,3.562552,54.702354,-3.276575
5,Germany,2,1,23,26,7.692308,3.846154,88.461538,2.154101,51.163818,10.447831
13,France,0,2,21,23,0.000000,8.695652,91.304348,1.905551,46.603354,1.888334
11,Canada,0,0,18,18,0.000000,0.000000,100.000000,1.491301,61.066692,-107.991707
14,Israel,2,0,16,18,11.111111,0.000000,88.888889,1.491301,30.812425,34.859476
18,Brazil,0,1,15,16,0.000000,6.250000,93.750000,1.325601,-10.333333,-53.200000
19,South Korea,2,0,11,13,15.384615,0.000000,84.615385,1.077051,36.638392,127.696119


In [13]:
# DF of Top 10 Countries with the greatest # of growing startups

print("Top 10 Countries with the greatest # of growing startups")

greatest_number_growing

Top 10 Countries with the greatest # of growing startups


,Country,Number of Growing Startups,Number of Declining Startups,Number of Stagnant Startups,Total Number of Startups,Percent of Growing Startups,Percent of Declining Startups,Percent of Stagnant Startups,Percent of World's Startups,Lat,Lng
1,United States,48,15,518,581,8.261618,2.581756,89.156627,48.135874,39.783730,-100.445882
4,India,7,5,47,59,11.864407,8.474576,79.661017,4.888152,22.351115,78.667743
0,China,4,1,144,149,2.684564,0.671141,96.644295,12.344656,35.000074,104.999927
3,United Kingdom,3,3,37,43,6.976744,6.976744,86.046512,3.562552,54.702354,-3.276575
2,Australia,2,1,5,8,25.000000,12.500000,62.500000,0.662800,-24.776109,134.755000
5,Germany,2,1,23,26,7.692308,3.846154,88.461538,2.154101,51.163818,10.447831
10,Estonia,2,0,3,5,40.000000,0.000000,60.000000,0.414250,58.752378,25.331908
14,Israel,2,0,16,18,11.111111,0.000000,88.888889,1.491301,30.812425,34.859476
19,South Korea,2,0,11,13,15.384615,0.000000,84.615385,1.077051,36.638392,127.696119
25,Singapore,2,0,11,13,15.384615,0.000000,84.615385,1.077051,1.357107,103.819499


In [15]:
# DF of Top 10 Countries with the greatest # of declining startups

print("Top 5 Countries with the greatest # of declining startups")

greatest_number_declining

Top 5 Countries with the greatest # of declining startups


,Country,Number of Growing Startups,Number of Declining Startups,Number of Stagnant Startups,Total Number of Startups,Percent of Growing Startups,Percent of Declining Startups,Percent of Stagnant Startups,Percent of World's Startups,Lat,Lng
1,United States,48,15,518,581,8.261618,2.581756,89.156627,48.135874,39.783730,-100.445882
4,India,7,5,47,59,11.864407,8.474576,79.661017,4.888152,22.351115,78.667743
3,United Kingdom,3,3,37,43,6.976744,6.976744,86.046512,3.562552,54.702354,-3.276575
13,France,0,2,21,23,0.000000,8.695652,91.304348,1.905551,46.603354,1.888334
0,China,4,1,144,149,2.684564,0.671141,96.644295,12.344656,35.000074,104.999927


In [16]:
# DF of Top 10 Countries with the greatest % of growing startups

print("Top 10 Countries with the greatest % of growing startups")

greatest_percent_growing

Top 10 Countries with the greatest % of growing startups


,Country,Number of Growing Startups,Number of Declining Startups,Number of Stagnant Startups,Total Number of Startups,Percent of Growing Startups,Percent of Declining Startups,Percent of Stagnant Startups,Percent of World's Startups,Lat,Lng
22,Lithuania,1,0,1,2,50.000000,0.000000,50.000000,0.165700,55.350000,23.750000
10,Estonia,2,0,3,5,40.000000,0.000000,60.000000,0.414250,58.752378,25.331908
6,Turkey,1,1,1,3,33.333333,33.333333,33.333333,0.248550,38.959759,34.924965
17,Belgium,1,0,2,3,33.333333,0.000000,66.666667,0.248550,50.640281,4.666715
2,Australia,2,1,5,8,25.000000,12.500000,62.500000,0.662800,-24.776109,134.755000
19,South Korea,2,0,11,13,15.384615,0.000000,84.615385,1.077051,36.638392,127.696119
25,Singapore,2,0,11,13,15.384615,0.000000,84.615385,1.077051,1.357107,103.819499
4,India,7,5,47,59,11.864407,8.474576,79.661017,4.888152,22.351115,78.667743
14,Israel,2,0,16,18,11.111111,0.000000,88.888889,1.491301,30.812425,34.859476
1,United States,48,15,518,581,8.261618,2.581756,89.156627,48.135874,39.783730,-100.445882


In [17]:
# DF of Top 10 Countries with the greatest % of declining startups

print("Top 10 Countries with the greatest % of declining startups")

greatest_percent_declining

Top 10 Countries with the greatest % of declining startups


,Country,Number of Growing Startups,Number of Declining Startups,Number of Stagnant Startups,Total Number of Startups,Percent of Growing Startups,Percent of Declining Startups,Percent of Stagnant Startups,Percent of World's Startups,Lat,Lng
6,Turkey,1,1,1,3,33.333333,33.333333,33.333333,0.248550,38.959759,34.924965
12,Netherlands,0,1,6,7,0.000000,14.285714,85.714286,0.579950,52.243498,5.634323
2,Australia,2,1,5,8,25.000000,12.500000,62.500000,0.662800,-24.776109,134.755000
13,France,0,2,21,23,0.000000,8.695652,91.304348,1.905551,46.603354,1.888334
4,India,7,5,47,59,11.864407,8.474576,79.661017,4.888152,22.351115,78.667743
3,United Kingdom,3,3,37,43,6.976744,6.976744,86.046512,3.562552,54.702354,-3.276575
18,Brazil,0,1,15,16,0.000000,6.250000,93.750000,1.325601,-10.333333,-53.200000
5,Germany,2,1,23,26,7.692308,3.846154,88.461538,2.154101,51.163818,10.447831
1,United States,48,15,518,581,8.261618,2.581756,89.156627,48.135874,39.783730,-100.445882
0,China,4,1,144,149,2.684564,0.671141,96.644295,12.344656,35.000074,104.999927


In [20]:
# DF of Top 10 Startups with the greatest CAGR

print("Top 10 Startups with the greatest CAGR")

top_startups

Top 10 Startups with the greatest CAGR


,Company,2022_Valuation,2023_Valuation,2024_Valuation,Date Joined,Country,City,Industry,Select Investors,Overall_Growth,CAGR (%),Lat,Lng
299,OpenAI,2.92,29.00,80.00,7/22/2019,United States,San Francisco,Artificial intelligence,Khosla Ventures,77.08,423.42,37.640314,-123.173825
819,Branch,1.05,4.00,4.00,3/1/2022,United States,Columbus,Fintech,"American Family Ventures, Greycroft, SignalFire",2.95,95.18,39.808694,-83.210180
403,Improbable,1.00,3.36,3.36,5/11/2017,United Kingdom,London,Other,"Andreessen Horowitz, SoftBank Group, Temasek H...",2.36,83.30,51.286760,-0.510375
1048,LayerZero Labs,1.00,3.00,3.00,3/30/2022,United States,New York,Internet software & services,"Andreessen Horowitz, FTX Ventures, Tiger Globa...",2.00,73.21,40.476578,-74.258843
952,Shield AI,1.00,2.30,2.80,8/24/2021,United States,San Diego,Artificial intelligence,"Andreessen Horowitz, Homebrew, Point72 Ventures",1.80,67.33,32.534798,-117.309816
933,Claroty,1.00,1.00,2.50,6/17/2021,United States,New York,Cybersecurity,"Bessemer Venture Partners, MoreVC, Team8",1.50,58.11,40.476578,-74.258843
206,VAST Data,3.70,3.70,9.10,4/16/2020,United States,New York,Data management & analytics,"Norwest Venture Partners, Goldman Sachs, Dell ...",5.40,56.83,40.476578,-74.258843
460,Hugging Face,2.00,2.00,4.50,5/9/2022,United States,New York,Artificial intelligence,"Betaworks Ventures, Addition, Lux Capital",2.50,50.00,40.476578,-74.258843
899,Skydio,1.00,2.20,2.20,2/25/2021,United States,Redwood City,"Supply chain, logistics, & delivery","Andreessen Horowitz, Andreessen Horowitz, Inst...",1.20,48.32,37.447172,-122.290823
1061,Glean,1.00,1.00,2.20,5/18/2022,United States,Palo Alto,Internet software & services,"General Catalyst, Kleiner Perkins Caufield & B...",1.20,48.32,37.287679,-122.202665


In [21]:
# DF of Top 10 Startups with the lowest CAGR

print("Top 10 Startups with the lowest CAGR")

bottom_startups

Top 10 Startups with the lowest CAGR


,Company,2022_Valuation,2023_Valuation,2024_Valuation,Date Joined,Country,City,Industry,Select Investors,Overall_Growth,CAGR (%),Lat,Lng
5,Checkout.com,40.00,11.00,11.00,5/2/2019,United Kingdom,London,Fintech,"Tiger Global Management, Insight Partners, DST...",-29.00,-47.56,51.286760,-0.510375
180,Patreon,4.00,1.35,1.35,9/2/2020,United States,San Francisco,Internet software & services,"Index Ventures, Thrive Capital, CRV",-2.65,-41.91,37.640314,-123.173825
304,Flink,2.85,1.07,1.07,12/1/2021,Germany,Berlin,E-commerce & direct-to-consumer,"Mubadala Capital, Bond, Prosus Ventures",-1.78,-38.73,52.338245,13.088345
48,OutSystems,9.50,4.30,4.30,6/5/2018,United States,Boston,Internet software & services,"KKR, ES Ventures, North Bridge Growth Equity",-5.20,-32.72,42.227911,-71.191244
19,Blockchain.com,14.00,14.00,7.00,2/17/2021,United Kingdom,London,Fintech,"Lightspeed Venture Partners, Google Ventures, ...",-7.00,-29.29,51.286760,-0.510375
301,Loft,2.90,2.90,1.46,1/3/2020,Brazil,Sao Paulo,E-commerce & direct-to-consumer,"Monashees+, Andreessen Horowitz, QED Investors",-1.44,-29.05,-24.007900,-46.826269
11,BYJU's,22.00,11.50,11.50,7/25/2017,India,Bengaluru,Edtech,"Tencent Holdings, Lightspeed India Partners, S...",-10.50,-27.70,12.816794,77.430082
260,Udaan,3.10,3.10,1.70,9/4/2018,India,Bengaluru,"Supply chain, logistics, & delivery","DST Global, Lightspeed Venture Partners, Micro...",-1.40,-25.95,12.816794,77.430082
4,Canva,40.00,40.00,25.40,1/8/2018,Australia,Surry Hills,Internet software & services,"Sequoia Capital China, Blackbird Ventures, Mat...",-14.60,-20.31,-33.892106,151.201846
2,SHEIN,100.00,66.00,66.00,7/3/2018,China,Shenzhen,E-commerce & direct-to-consumer,"Tiger Global Management, Sequoia Capital China...",-34.00,-18.76,22.287600,113.679356


In [6]:
country_df["Ratio of Growing to Declining"] = country_df["Number of Growing Startups"]/(country_df["Number of Declining Startups"] + country_df["Number of Stagnant Startups"])

country_df

,Country,Number of Growing Startups,Number of Declining Startups,Number of Stagnant Startups,Total Number of Startups,Percent of Growing Startups,Percent of Declining Startups,Percent of Stagnant Startups,Percent of World's Startups,Lat,Lng,Ratio of Growing to Declining
0,China,4,1,144,149,2.684564,0.671141,96.644295,12.344656,35.000074,104.999927,0.027586
1,United States,48,15,518,581,8.261618,2.581756,89.156627,48.135874,39.783730,-100.445882,0.090056
2,Australia,2,1,5,8,25.000000,12.500000,62.500000,0.662800,-24.776109,134.755000,0.333333
3,United Kingdom,3,3,37,43,6.976744,6.976744,86.046512,3.562552,54.702354,-3.276575,0.075000
4,India,7,5,47,59,11.864407,8.474576,79.661017,4.888152,22.351115,78.667743,0.134615
5,Germany,2,1,23,26,7.692308,3.846154,88.461538,2.154101,51.163818,10.447831,0.083333
6,Turkey,1,1,1,3,33.333333,33.333333,33.333333,0.248550,38.959759,34.924965,0.500000
7,Seychelles,0,0,1,1,0.000000,0.000000,100.000000,0.082850,-4.657498,55.454015,0.000000
8,Sweden,0,0,5,5,0.000000,0.000000,100.000000,0.414250,59.674971,14.520858,0.000000
9,Mexico,0,0,8,8,0.000000,0.000000,100.000000,0.662800,23.658512,-102.007710,0.000000


Analysis of Map Visualizations

These maps collectively highlight the global distribution and trends of unicorn startups. This could provide valuable insights for investors interested in the startup landscape. 

It is clear that the United States and China dominate the startup ecosystem with the largest share and number of unicorn startups. The U.S. contains 43.14% of the world's unicorn startups, while China contains 12.34%. 

Countries with a larger volume of startups, including the U.S., China, and India, demonstrate greater stability in terms of the growth and decline of unicorn startups. Roughly 8.26% of U.S. startups are growing and 2.58% are declining; 2.68% of startups in China are growing while 0.67% are declining. 11.86% of startups in India are growing and 8.47% are declining. Conversely, these numbers are much higher in countries with smaller startup ecosystems. They tend to have either a higher percentage of growing startups or a higher percentage of declining startups within their respective countries. This suggests that countries with a smaller startup ecosystem experience greater volatility and less stability. Infrastructure and political and economic conditions may have a more pronounced impact on whether these countries experience high-growth or high-risk opportunities. 